In [1]:
import pandas as pd
import boto3
import re
import json
import os
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv
from tokencost import calculate_prompt_cost, calculate_completion_cost

COST = 0

bedrock_runtime = boto3.client("bedrock-runtime", region_name="us-east-1")

ERROR:root:Failed to update token costs. Using static costs.
/Users/krystianpietrzak/Documents/ML/Phi3-FT/.venv/lib/python3.9/site-packages/tokencost/constants.py:61: RuntimeWarning: coroutine 'update_token_costs' was never awaited
  TOKEN_COSTS = TOKEN_COSTS_STATIC


In [2]:
df = pd.read_csv('train.csv')

In [3]:
def claude_inference(prompt):

    prompt_config = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 4096,
        "temperature":0.1,
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                ],
            }
        ],
    }

    body = json.dumps(prompt_config)

    model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
    accept = "application/json"
    contentType = "application/json"

    response = bedrock_runtime.invoke_model(
        body=body, modelId=model_id, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    results = response_body.get("content")[0].get("text")

    total_cost = float((len(results)/4/1000)* 0.003+ (len(prompt)/4/1000)*0.015)
    print(f'{np.round(total_cost,3)}$')
    
    return results, total_cost

In [4]:
def gpt_inference(prompt):
    # Load API key from .env file
    load_dotenv()
    api_key = os.getenv("OPENAI_API_KEY")
    client = OpenAI(api_key=api_key)
    
    if api_key is None:
        raise ValueError("API key not found. Please set the OPENAI_API_KEY environment variable in the .env file.")
    
    completion = client.chat.completions.create(
                                            model="gpt-4o",
                                            temperature=0.1,
                                            max_tokens=4096,
                                            messages=[
                                                {"role": "user", "content": prompt}
                                            ]
                                            )
    
    response = completion.choices[0].message.content

    prompt_cost = calculate_prompt_cost(prompt, "gpt-4o")
    completion_cost = calculate_completion_cost(response, "gpt-4o")
    total_cost = float(prompt_cost + completion_cost)

    print(f"{np.round(total_cost,3)}$")

    return response, total_cost

In [5]:
def simulate_user(row, model, complete=True):
    user = row['chat']
    user = re.findall(r'^(USER: .+?|ASSISTANT: .+?|FUNCTION RESPONSE: .+?)(?=\n[A-Z\s]+: |\Z)', user, re.DOTALL | re.MULTILINE)[0]
    system = row['system']
    if complete:
        prompt = f'''
                 Your task is to simulate a user question to the personal AI assistant that has an access to the API call given below:
                {system}
                Your simulated question must resemble a question that a human might ask to an AI assistant in a natural environment. 
                Your simulated question must contain all of the required parameters.
                Your response needs to start with USER:'''
    else:
        prompt = f'''
                 Your task is to simulate a user question to the personal AI assistant that has an access to the API call given below:
                {system}
                Your simulated question must resemble a question that a human might ask to an AI assistant in a natural environment. 
                Your simulated question MUST NOT contain all of the required api parameters. It has to be incomplete, contain only some api parameters or none.
                Your response needs to start with USER:'''
    if model == 'gpt':
        text, cost = gpt_inference(prompt)
    elif model == 'claude':
        text, cost = claude_inference(prompt)
    global COST
    COST += cost
    print(text)
    return text


df['gpt_complete'] = df.head(5).apply(simulate_user, axis=1, args=('gpt',))
df['gpt_incomplete'] = df.head(5).apply(simulate_user, axis=1, args=('gpt',False))
df['claude_complete'] = df.head(5).apply(simulate_user, axis=1, args=('claude',))
df['claude_incomplete'] = df.head(5).apply(simulate_user, axis=1, args=('claude',False))

0.001$
USER: What's the current exchange rate from USD to EUR?
0.001$
USER: Can you get me the latest news headlines from the United States?
0.003$
USER: Hi, can you help me create a new task? I need to set a reminder to submit my project report. The due date is October 15, 2023, and it's really important, so please set the priority to high.
0.001$
USER: Can you generate a random password for me that's 12 characters long, includes numbers, and has special characters?
0.001$
USER: Can you help me find the median of the following numbers: 3, 7, 2, 9, 5?
0.001$
USER: What's the current exchange rate from USD?
0.001$
USER: Can you give me the latest news headlines from the US?
0.002$
USER: Can you help me create a new task titled "Finish project report"?
0.001$
USER: Can you help me create a strong password? Maybe something with special characters?
0.001$
USER: Hey, can you help me find the median of these numbers: 5, 3, 8, 2, and 7?
0.004$
USER: Hey there! I'm planning a trip to Japan soo

In [6]:
def assistant_ask(row, model):
    if model == 'gpt':
        user = row['gpt_incomplete']
    elif model == 'claude':
        user = row['claude_incomplete']
    system = row['system']
    prompt = f'''
            Here is the user question that is missing some parameters required to perform an API call: "{user}"
            As an assistant, your task is to ask a clarifing question to the user to give missing parameters 
            so that you will be able to perform the API call given below:
            {system}
            Your clarifing question needs to start with ASSISTANT:'''
    if model == 'gpt':
        text, cost = gpt_inference(prompt)
    elif model == 'claude':
        text, cost = claude_inference(prompt)
    global COST
    COST += cost
    print(text)
    return text

df['gpt_incomplete_ask'] = df.head(5).apply(assistant_ask, axis=1, args=('gpt',))
df['claude_incomplete_ask'] = df.head(5).apply(assistant_ask, axis=1, args=('claude',))

0.002$
ASSISTANT: Could you please specify the target currency you would like to convert USD to? For example, EUR, GBP, JPY, etc.
0.001$
ASSISTANT: Could you please specify the country for which you would like to fetch the latest news headlines?
0.003$
ASSISTANT: I'd be happy to help you create a new task titled "Finish project report." Could you please provide the due date for the task and its priority (low, medium, or high)?
0.002$
ASSISTANT: Sure, I'd be happy to help you create a strong password! Could you please specify the desired length of the password and let me know if you would like to include numbers in addition to special characters?
0.002$
ASSISTANT: Could you please confirm if the list of numbers you provided (5, 3, 8, 2, and 7) is complete, or are there any additional numbers you would like to include?
0.005$
ASSISTANT: Thank you for your question about the exchange rate. To provide you with the most accurate information, I need to confirm a few details. While you've men

In [7]:
def simulate_clarification(row, model):
    if model == 'gpt':
        user = row['gpt_incomplete']
        question = row['gpt_incomplete_ask']
    elif model == 'claude':
        user = row['claude_incomplete']
        question = row['claude_incomplete_ask']
    system = row['system']
    prompt = f'''
            Here is the first user question that is missing some parameters required to perform an API call: "{user}"
            Here is the clarification question that the AI assistant asked: {question}
            Your task is to simulate a user response to this question so that it contains all the required parameters to the following API call:
            {system}
            Remember that maybe some of the parameters were included in the first user question!
            Return the simulated user response only! Your simulated user response must start with USER:'''
    if model == 'gpt':
        text, cost = gpt_inference(prompt)
    elif model == 'claude':
        text, cost = claude_inference(prompt)
    global COST
    COST += cost
    print(text)
    return text

df['gpt_incomplete_clarification'] = df.head(5).apply(simulate_clarification, axis=1, args=('gpt',))
df['claude_incomplete_clarification'] = df.head(5).apply(simulate_clarification, axis=1, args=('claude',))

0.002$
USER: I would like to convert USD to EUR.
0.001$
USER: I would like to fetch the latest news headlines from the US.
0.003$
USER: The due date for the task is 2023-10-15, and the priority is high.
0.002$
USER: I would like the password to be 12 characters long and yes, please include numbers along with special characters.
0.002$
USER: Yes, the list of numbers I provided (5, 3, 8, 2, and 7) is complete.
0.007$
USER: I'd like to know how many euros I can get for 1 US dollar. So the base currency would be US dollars and the target currency would be euros.
0.005$
USER: I'm interested in news from the United States. Could you please provide me with the latest headlines from there?
0.01$
USER: Sure, I can provide those details:

1. The title of the task should be "Finish Project Presentation".
2. Next Friday's date is May 26, 2023.
3. I'd rate the priority of this task as high, given its importance.

Is there anything else you need to create the task?
0.007$
USER: I'd like the password

In [8]:
df['gpt_incomplete_conversation'] = df['gpt_incomplete'] + '\n' + df['gpt_incomplete_ask'] + '\n' + df['gpt_incomplete_clarification']
df['claude_incomplete_conversation'] = df['claude_incomplete'] + '\n' + df['claude_incomplete_ask'] + '\n' + df['claude_incomplete_clarification']

In [9]:
index = 4
print('GPT CONVERSATION')
print(df.iloc[index]['gpt_incomplete_conversation'])
print('\n\n')
print('CLAUDE CONVERSATION')
print(df.iloc[index]['claude_incomplete_conversation'])

GPT CONVERSATION
USER: Hey, can you help me find the median of these numbers: 5, 3, 8, 2, and 7?
ASSISTANT: Could you please confirm if the list of numbers you provided (5, 3, 8, 2, and 7) is complete, or are there any additional numbers you would like to include?
USER: Yes, the list of numbers I provided (5, 3, 8, 2, and 7) is complete.



CLAUDE CONVERSATION
USER: Hey there! I've got a bunch of test scores from my class, and I'm trying to figure out what the middle score is. Can you help me find that?
ASSISTANT: Certainly! I'd be happy to help you find the middle score, which is also known as the median. To calculate this accurately, I'll need some more information. Could you please provide me with the list of test scores from your class? It would be best if you could list all the scores, separated by commas. For example: "75, 82, 90, 68, 95". Once I have the complete list of scores, I'll be able to calculate the median for you.
USER: Oh, of course! Sorry I didn't provide that inform

In [10]:
def simulate_function_call_complete(row, model):
    if model == 'gpt':
        user = row['gpt_complete']
    elif model == 'claude':
        user = row['claude_complete']
    system = row['system']
    prompt = f'''{system}
                 {user}
                 Return a function call in the format: 
                 <function_call>function_name:{{pamarameters}}</function_call> and nothing else!
                 '''
    print(prompt)
    if model == 'gpt':
        text, cost = gpt_inference(prompt)
    elif model == 'claude':
        text, cost = claude_inference(prompt)
    global COST
    COST += cost
    print(text)
    return text

df['gpt_complete_fc'] = df.head(5).apply(simulate_function_call_complete, axis=1, args=('gpt',))
df['claude_complete_fc'] = df.head(5).apply(simulate_function_call_complete, axis=1, args=('claude',))

SYSTEM: You are a helpful assistant with access to the following functions. Use them if required -
{
    "name": "get_exchange_rate",
    "description": "Get the exchange rate between two currencies",
    "parameters": {
        "type": "object",
        "properties": {
            "base_currency": {
                "type": "string",
                "description": "The currency to convert from"
            },
            "target_currency": {
                "type": "string",
                "description": "The currency to convert to"
            }
        },
        "required": [
            "base_currency",
            "target_currency"
        ]
    }
}

                 USER: What's the current exchange rate from USD to EUR?
                 Return a function call in the format: 
                 <function_call>function_name:{pamarameters}</function_call> and nothing else!
                 
0.001$
<function_call>get_exchange_rate:{"base_currency":"USD","target_currency":"EUR"}</func

In [12]:
def simulate_function_call_incomplete(row, model):
    if model == 'gpt':
        conversation = row['gpt_incomplete_conversation']
    elif model == 'claude':
        conversation = row['claude_incomplete_conversation']
    system = row['system']
    prompt = f'''{system}
                 {conversation}
                 Return a function call in the format: 
                 <function_call>function_name:{{pamarameters}}</function_call> and nothing else!
                 '''
    print(prompt)
    if model == 'gpt':
        text, cost = gpt_inference(prompt)
    elif model == 'claude':
        text, cost = claude_inference(prompt)
    global COST
    COST += cost
    print(text)
    return text

df['gpt_incomplete_fc'] = df.head(5).apply(simulate_function_call_incomplete, axis=1, args=('gpt',))
df['claude_incomplete_fc'] = df.head(5).apply(simulate_function_call_incomplete, axis=1, args=('claude',))

SYSTEM: You are a helpful assistant with access to the following functions. Use them if required -
{
    "name": "get_exchange_rate",
    "description": "Get the exchange rate between two currencies",
    "parameters": {
        "type": "object",
        "properties": {
            "base_currency": {
                "type": "string",
                "description": "The currency to convert from"
            },
            "target_currency": {
                "type": "string",
                "description": "The currency to convert to"
            }
        },
        "required": [
            "base_currency",
            "target_currency"
        ]
    }
}

                 USER: What's the current exchange rate from USD?
ASSISTANT: Could you please specify the target currency you would like to convert USD to? For example, EUR, GBP, JPY, etc.
USER: I would like to convert USD to EUR.
                 Return a function call in the format: 
                 <function_call>function_name:{p

In [ ]:
index = 3
print('GPT CONVERSATION')
print(df.iloc[index]['gpt_complete'])
print(df.iloc[index]['system'])
print(df.iloc[index]['gpt_complete_fc'])
print('\n\n')
print('CLAUDE CONVERSATION')
print(df.iloc[index]['claude_complete'])
print(df.iloc[index]['system'])
print(df.iloc[index]['claude_complete_fc'])

GPT CONVERSATION
USER: Hey, can you help me generate a password? I need it to be 12 characters long, and it should include both numbers and special characters.
SYSTEM: You are a helpful assistant with access to the following functions. Use them if required -
{
    "name": "generate_password",
    "description": "Generate a random password with specified criteria",
    "parameters": {
        "type": "object",
        "properties": {
            "length": {
                "type": "integer",
                "description": "The length of the password"
            },
            "include_numbers": {
                "type": "boolean",
                "description": "Include numbers in the password"
            },
            "include_special_characters": {
                "type": "boolean",
                "description": "Include special characters in the password"
            }
        },
        "required": [
            "length"
        ]
    }
}

<function_call>generate_password:{"leng

In [ ]:
index = 3
print('GPT CONVERSATION')
print(df.iloc[index]['gpt_incomplete_conversation'])
print(df.iloc[index]['system'])
print(df.iloc[index]['gpt_incomplete_fc'])
print('\n\n')
print('CLAUDE CONVERSATION')
print(df.iloc[index]['claude_incomplete_conversation'])
print(df.iloc[index]['system'])
print(df.iloc[index]['claude_incomplete_fc'])

GPT CONVERSATION
USER: Can you help me create a strong password? Maybe around 12 characters long?
ASSISTANT: Sure, I can help with that! Do you want the password to include numbers and special characters as well?
USER: Yes, please include both numbers and special characters.
SYSTEM: You are a helpful assistant with access to the following functions. Use them if required -
{
    "name": "generate_password",
    "description": "Generate a random password with specified criteria",
    "parameters": {
        "type": "object",
        "properties": {
            "length": {
                "type": "integer",
                "description": "The length of the password"
            },
            "include_numbers": {
                "type": "boolean",
                "description": "Include numbers in the password"
            },
            "include_special_characters": {
                "type": "boolean",
                "description": "Include special characters in the password"
          

In [13]:
df.head().to_csv('train_synthetic.csv')

In [19]:
df.iloc[1]['chat']

'USER: Can you tell me the latest news headlines for the United States?\n\n\nASSISTANT: <functioncall> {"name": "get_news_headlines", "arguments": \'{"country": "United States"}\'} <|endoftext|>\n\n\nFUNCTION RESPONSE: {"headlines": ["Biden announces new vaccine mandates", "Hurricane Ida devastates Louisiana", "Apple unveils new iPhone", "NASA\'s Perseverance rover collects first Mars rock sample"]}\n\n\nASSISTANT: Here are the latest news headlines for the United States:\n1. Biden announces new vaccine mandates\n2. Hurricane Ida devastates Louisiana\n3. Apple unveils new iPhone\n4. NASA\'s Perseverance rover collects first Mars rock sample <|endoftext|>\n\n\nUSER: That\'s interesting. What about the news in France?\n\n\nASSISTANT: <functioncall> {"name": "get_news_headlines", "arguments": \'{"country": "France"}\'} <|endoftext|>\n\n\nFUNCTION RESPONSE: {"headlines": ["France recalls ambassadors to US and Australia", "French election: Macron\'s party braces for tough fight", "Louvre Mu

In [24]:
(COST / 5) * 112960 / 40 / 3 / 2

20.186422666666672

In [26]:
112960 / 40 * 2

5648.0